# Part 7: 평가와 프로덕션 — GraphRAG 실무 적용 가이드

**소요시간**: 1시간 | **난이도**: ★★★★ | **시리즈 최종편**

---

Part 1~6에서 구축한 GraphRAG 시스템의 **품질을 정량 평가**하고,  
**프로덕션 배포를 위한 최적화와 운영 체크리스트**를 완성합니다.

| 섹션 | 내용 | 시간 |
|------|------|------|
| 1 | 환경 설정 | 5분 |
| 2 | RAGAS 4대 메트릭 이해 | 10분 |
| 3 | 질문 난이도별 평가 | 15분 |
| 4 | Vector RAG vs GraphRAG 비교 | 10분 |
| 5 | Neo4j 성능 최적화 | 10분 |
| 6 | 프로덕션 체크리스트 | 5분 |
| 7 | 최종 아키텍처 요약 | 3분 |
| 8 | 연습 문제 | 2분 |

> **이 노트북은 7개 시리즈의 마지막입니다.**  
> Part 1의 회색 아키텍처가 이제 전부 컬러로 채워집니다.

---
## 1. 환경 설정

RAGAS 평가 프레임워크, Neo4j, OpenAI를 연결하고 평가 질문 데이터를 로드합니다.

In [ ]:
# ============================================================
# 1-1. 패키지 임포트
# ============================================================
import os
import json
import time
import warnings
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

from dotenv import load_dotenv
from neo4j import GraphDatabase
from openai import OpenAI

# RAGAS 평가 메트릭
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)

warnings.filterwarnings('ignore')

# 한글 폰트 설정 (matplotlib)
matplotlib.rcParams['font.family'] = 'AppleGothic'  # macOS
# matplotlib.rcParams['font.family'] = 'Malgun Gothic'  # Windows
matplotlib.rcParams['axes.unicode_minus'] = False

print("패키지 로드 완료")

In [ ]:
# ============================================================
# 1-2. 환경변수 로드 및 클라이언트 초기화
# ============================================================
load_dotenv()

# OpenAI 클라이언트
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Neo4j 연결
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USER = os.getenv("NEO4J_USER", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "graphrag2024")

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# 연결 테스트
with driver.session() as session:
    result = session.run("RETURN 1 AS test")
    assert result.single()["test"] == 1
    print("Neo4j 연결 성공")

print(f"OpenAI API 키: ...{os.getenv('OPENAI_API_KEY', 'NOT_SET')[-4:]}")
print(f"Neo4j URI: {NEO4J_URI}")

In [ ]:
# ============================================================
# 1-3. 평가 질문 데이터 로드
# ============================================================
# data/eval_questions.json 파일이 있으면 로드, 없으면 내장 데이터 사용
eval_path = Path("data/eval_questions.json")

if eval_path.exists():
    with open(eval_path, encoding="utf-8") as f:
        eval_data = json.load(f)
    print(f"평가 질문 로드 완료: {len(eval_data)}개")
else:
    # 내장 평가 데이터셋 (30개: Easy 10, Medium 10, Hard 10)
    eval_data = [
        # === Easy (1-hop) 질문 10개 ===
        {"question": "삼성전자가 양산을 시작한 차세대 메모리는?",
         "ground_truth": "HBM4", "difficulty": "easy", "hops": 1},
        {"question": "네이버가 출시한 기업용 AI 에이전트 이름은?",
         "ground_truth": "CLOVA for Enterprise", "difficulty": "easy", "hops": 1},
        {"question": "SK하이닉스가 인수한 인텔 사업부는?",
         "ground_truth": "NAND 플래시 사업부 (솔리다임)", "difficulty": "easy", "hops": 1},
        {"question": "카카오가 투자한 자율주행 스타트업은?",
         "ground_truth": "오토노머스에이투지", "difficulty": "easy", "hops": 1},
        {"question": "LG에너지솔루션이 착공한 미국 공장 위치는?",
         "ground_truth": "애리조나주 퀸크릭", "difficulty": "easy", "hops": 1},
        {"question": "현대자동차가 스마트팩토리에 투입한 로봇 이름은?",
         "ground_truth": "스팟(Spot)", "difficulty": "easy", "hops": 1},
        {"question": "카카오뱅크 AI 어드바이저가 활용한 모델은?",
         "ground_truth": "GPT-4o", "difficulty": "easy", "hops": 1},
        {"question": "LG AI연구원이 공개한 의료 AI 모델 이름은?",
         "ground_truth": "엑사원 메드(EXAONE Med)", "difficulty": "easy", "hops": 1},
        {"question": "쿠팡이 도입 예정인 물류 로봇 수량은?",
         "ground_truth": "500대 이상", "difficulty": "easy", "hops": 1},
        {"question": "HBM4가 탑재될 NVIDIA GPU 이름은?",
         "ground_truth": "Blackwell Ultra", "difficulty": "easy", "hops": 1},

        # === Medium (2-hop) 질문 10개 ===
        {"question": "네이버와 클라우드 인프라 협력을 한 기업이 속한 그룹은?",
         "ground_truth": "삼성그룹 (삼성SDS)", "difficulty": "medium", "hops": 2},
        {"question": "삼성전자 HBM4가 탑재될 GPU를 만드는 회사의 본사 위치는?",
         "ground_truth": "미국 산타클라라 (NVIDIA)", "difficulty": "medium", "hops": 2},
        {"question": "카카오가 투자한 자율주행 기업의 기술 레벨은?",
         "ground_truth": "레벨4", "difficulty": "medium", "hops": 2},
        {"question": "LG에너지솔루션 배터리를 공급받을 자동차 회사의 CEO는?",
         "ground_truth": "일론 머스크 (테슬라)", "difficulty": "medium", "hops": 2},
        {"question": "현대자동차가 인수한 로봇 기업의 대표 제품은?",
         "ground_truth": "스팟(Spot) - 보스턴다이내믹스", "difficulty": "medium", "hops": 2},
        {"question": "엑사원 메드를 공동 개발한 병원들이 위치한 도시는?",
         "ground_truth": "서울 (서울대병원, 아산병원)", "difficulty": "medium", "hops": 2},
        {"question": "쿠팡과 파트너십을 맺은 물류 로봇 기업의 국적은?",
         "ground_truth": "중국 (긱플러스)", "difficulty": "medium", "hops": 2},
        {"question": "온디바이스 AI MOU를 체결한 두 기업의 대표 직함은?",
         "ground_truth": "이재용 회장, 최수연 대표", "difficulty": "medium", "hops": 2},
        {"question": "NAND 사업부 인수 금액과 인수한 기업의 CEO 이름은?",
         "ground_truth": "90억 달러, 곽노정", "difficulty": "medium", "hops": 2},
        {"question": "카카오뱅크가 받은 금융위원회 지정과 가입자 수는?",
         "ground_truth": "혁신금융서비스 지정, 50만 명", "difficulty": "medium", "hops": 2},

        # === Hard (3-hop / Multi-hop) 질문 10개 ===
        {"question": "삼성전자와 네이버가 공동 개발한 기술이 적용될 기기 시리즈와 출시 연도는?",
         "ground_truth": "갤럭시 S26, 2025년", "difficulty": "hard", "hops": 3},
        {"question": "HBM4를 생산하는 기업과 HBM 시장에서 경쟁하는 기업이 공통으로 속한 산업 분류는?",
         "ground_truth": "반도체 (삼성전자, SK하이닉스)", "difficulty": "hard", "hops": 3},
        {"question": "보스턴다이내믹스를 인수한 자동차 기업이 로봇을 투입한 공장 위치와 로봇 수량은?",
         "ground_truth": "울산 공장, 스팟 20대", "difficulty": "hard", "hops": 3},
        {"question": "AI 에이전트를 출시한 기업과 온디바이스 MOU를 체결한 상대 기업의 반도체 제품명은?",
         "ground_truth": "네이버-삼성전자 MOU, HBM4", "difficulty": "hard", "hops": 3},
        {"question": "카카오 그룹에서 자율주행과 AI 금융 서비스를 각각 담당하는 자회사는?",
         "ground_truth": "카카오모빌리티, 카카오뱅크", "difficulty": "hard", "hops": 3},
        {"question": "LG 그룹에서 배터리와 AI를 담당하는 두 자회사의 해외 진출 지역은?",
         "ground_truth": "LG에너지솔루션(미국 애리조나), LG AI연구원(글로벌 의료 AI)",
         "difficulty": "hard", "hops": 3},
        {"question": "테슬라에 배터리를 공급하는 기업의 CEO와 투자 금액, 그리고 IRA 보조금 관련 전략은?",
         "ground_truth": "김동명, 55억 달러, IRA 보조금 극대화", "difficulty": "hard", "hops": 3},
        {"question": "GPT-4o를 활용하는 두 서비스(금융, 의료)의 공통점과 차이점은?",
         "ground_truth": "카카오뱅크 AI 어드바이저(금융) vs 엑사원 메드(의료) — 둘 다 LLM 활용, 도메인 특화",
         "difficulty": "hard", "hops": 3},
        {"question": "물류 자동화에 로봇을 도입한 두 기업의 예상 효과를 비교하면?",
         "ground_truth": "쿠팡(처리속도 40% 향상, 인건비 30% 절감) vs 현대자동차(설비점검, 안전모니터링 자동화)",
         "difficulty": "hard", "hops": 3},
        {"question": "반도체 기업 두 곳의 CEO, 주력 제품, 경쟁 관계를 종합적으로 설명하면?",
         "ground_truth": "삼성전자(전영현 부회장, HBM4) vs SK하이닉스(곽노정 CEO, NAND+HBM) — HBM 시장 경쟁",
         "difficulty": "hard", "hops": 3}
    ]
    print(f"내장 평가 질문 로드 완료: {len(eval_data)}개")

# 난이도별 분류
easy_qs = [q for q in eval_data if q["difficulty"] == "easy"]
medium_qs = [q for q in eval_data if q["difficulty"] == "medium"]
hard_qs = [q for q in eval_data if q["difficulty"] == "hard"]

print(f"\n난이도 분포:")
print(f"  Easy  (1-hop): {len(easy_qs)}개")
print(f"  Medium(2-hop): {len(medium_qs)}개")
print(f"  Hard  (3-hop): {len(hard_qs)}개")

---
## 2. RAGAS 4대 메트릭 이해

RAGAS(Retrieval Augmented Generation Assessment)는 RAG 시스템 평가의 **사실상 표준**입니다.  
4가지 메트릭으로 시스템 품질을 다각도로 측정합니다.

| 메트릭 | 질문 | 측정 대상 | 범위 |
|--------|------|-----------|------|
| **Faithfulness** | 답변이 검색 결과에 근거하는가? | 환각(Hallucination) 방지 | 0~1 |
| **Answer Relevancy** | 답변이 질문에 적절한가? | 답변 관련성 | 0~1 |
| **Context Precision** | 검색된 문맥이 정확한가? | 검색 정밀도 | 0~1 |
| **Context Recall** | 필요한 정보가 모두 검색됐는가? | 검색 재현율 | 0~1 |

### 메트릭 해석 가이드

- **Faithfulness 낮음** → LLM이 검색 결과 없이 답변을 지어내고 있음 (환각)
- **Answer Relevancy 낮음** → 질문과 무관한 답변 생성 (프롬프트 개선 필요)
- **Context Precision 낮음** → 불필요한 문맥이 많이 검색됨 (검색 필터링 필요)
- **Context Recall 낮음** → 필요한 정보가 검색되지 않음 (인덱스 / 검색 전략 개선)

In [ ]:
# ============================================================
# 2-1. 헬퍼 함수: Neo4j에서 서브그래프 검색 (GraphRAG 방식)
# ============================================================
def graph_search(question: str, depth: int = 2) -> list[str]:
    """질문에서 키워드를 추출하고 Neo4j에서 관련 서브그래프를 검색합니다.
    
    Part 6에서 구축한 GraphRAG 파이프라인을 재사용합니다.
    - Text2Cypher 또는 키워드 기반 서브그래프 탐색
    - depth만큼 이웃 노드를 확장
    """
    # 1단계: LLM으로 질문에서 핵심 엔티티 추출
    extraction = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "질문에서 핵심 엔티티(회사명, 인물명, 제품명)를 JSON 배열로 추출하세요. 예: [\"삼성전자\", \"HBM4\"]"},
            {"role": "user", "content": question}
        ],
        temperature=0
    )
    
    try:
        entities = json.loads(extraction.choices[0].message.content)
    except json.JSONDecodeError:
        entities = [question.split()[0]]  # 폴백: 첫 단어
    
    # 2단계: Neo4j에서 엔티티 기반 서브그래프 검색
    contexts = []
    with driver.session() as session:
        for entity in entities[:3]:  # 최대 3개 엔티티
            # 퍼지 매칭으로 노드 검색 후 depth만큼 확장
            query = """
            MATCH (n)
            WHERE any(prop IN keys(n) WHERE toString(n[prop]) CONTAINS $entity)
            OPTIONAL MATCH path = (n)-[*1..{depth}]-(m)
            WITH n, collect(DISTINCT m) AS neighbors,
                 collect(DISTINCT relationships(path)) AS rels
            RETURN n, neighbors, rels
            LIMIT 5
            """.replace("{depth}", str(depth))
            
            try:
                result = session.run(query, entity=entity)
                for record in result:
                    node = record["n"]
                    # 노드 정보를 텍스트로 변환
                    node_text = f"[{':'.join(node.labels)}] {dict(node)}"
                    contexts.append(node_text)
                    
                    # 이웃 노드 정보 추가
                    for neighbor in record["neighbors"]:
                        if neighbor:
                            nb_text = f"  -> [{':'.join(neighbor.labels)}] {dict(neighbor)}"
                            contexts.append(nb_text)
            except Exception as e:
                contexts.append(f"검색 오류: {e}")
    
    return contexts if contexts else ["관련 정보를 찾을 수 없습니다."]


def vector_search(question: str, top_k: int = 5) -> list[str]:
    """벡터 유사도 기반 검색 (Vector-only RAG 시뮬레이션).
    
    Neo4j 벡터 인덱스 또는 단순 텍스트 매칭으로 문서를 검색합니다.
    GraphRAG와의 비교를 위해 그래프 탐색 없이 단일 문서만 반환합니다.
    """
    contexts = []
    with driver.session() as session:
        # 벡터 인덱스가 없는 경우 텍스트 매칭으로 폴백
        try:
            # 풀텍스트 인덱스 활용 시도
            result = session.run("""
                CALL db.index.fulltext.queryNodes('fulltext_index', $query)
                YIELD node, score
                RETURN node, score
                ORDER BY score DESC
                LIMIT $top_k
            """, query=question, top_k=top_k)
            
            for record in result:
                node = record["node"]
                contexts.append(f"[{':'.join(node.labels)}] {dict(node)} (score: {record['score']:.3f})")
        except Exception:
            # 풀텍스트 인덱스 없으면 단순 CONTAINS 검색
            keywords = question.replace("?", "").replace("은", "").replace("는", "").split()[:3]
            for kw in keywords:
                result = session.run("""
                    MATCH (n)
                    WHERE any(prop IN keys(n) WHERE toString(n[prop]) CONTAINS $kw)
                    RETURN n LIMIT 3
                """, kw=kw)
                for record in result:
                    node = record["n"]
                    contexts.append(f"[{':'.join(node.labels)}] {dict(node)}")
    
    # 그래프 탐색 없이 개별 노드만 반환 (Vector RAG 특성)
    return contexts[:top_k] if contexts else ["관련 정보를 찾을 수 없습니다."]


print("검색 함수 정의 완료")
print("  - graph_search(): 서브그래프 탐색 (GraphRAG)")
print("  - vector_search(): 단일 노드 매칭 (Vector RAG 시뮬레이션)")

In [ ]:
# ============================================================
# 2-2. RAG 답변 생성 함수
# ============================================================
def generate_answer(question: str, contexts: list[str], model: str = "gpt-4o-mini") -> str:
    """검색된 문맥을 바탕으로 LLM이 답변을 생성합니다."""
    context_text = "\n".join(contexts)
    
    response = openai_client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": (
                "주어진 문맥 정보만을 사용하여 질문에 답변하세요.\n"
                "문맥에 없는 정보는 '정보가 부족합니다'라고 답하세요.\n"
                "한국어로 간결하게 답변하세요."
            )},
            {"role": "user", "content": f"문맥:\n{context_text}\n\n질문: {question}"}
        ],
        temperature=0
    )
    return response.choices[0].message.content

print("답변 생성 함수 정의 완료")

In [ ]:
# ============================================================
# 2-3. RAGAS 메트릭 개별 계산 예시
# ============================================================
# 단일 질문에 대해 각 메트릭을 개별 확인

sample_q = eval_data[0]  # Easy 질문 1개
print(f"샘플 질문: {sample_q['question']}")
print(f"정답: {sample_q['ground_truth']}")
print(f"난이도: {sample_q['difficulty']} ({sample_q['hops']}-hop)")
print("---")

# GraphRAG로 검색 + 답변 생성
sample_contexts = graph_search(sample_q["question"])
sample_answer = generate_answer(sample_q["question"], sample_contexts)

print(f"검색된 문맥 수: {len(sample_contexts)}")
print(f"생성된 답변: {sample_answer}")
print("---")

# RAGAS Dataset 형식으로 변환
from datasets import Dataset

sample_dataset = Dataset.from_dict({
    "question": [sample_q["question"]],
    "answer": [sample_answer],
    "contexts": [sample_contexts],
    "ground_truth": [sample_q["ground_truth"]]
})

# 각 메트릭 개별 평가
print("\n=== RAGAS 4대 메트릭 개별 결과 ===")

for metric_name, metric in [
    ("Faithfulness", faithfulness),
    ("Answer Relevancy", answer_relevancy),
    ("Context Precision", context_precision),
    ("Context Recall", context_recall)
]:
    try:
        result = evaluate(sample_dataset, metrics=[metric])
        score = list(result.values())[0]
        bar = "*" * int(score * 20)
        print(f"  {metric_name:20s}: {score:.4f} |{bar}")
    except Exception as e:
        print(f"  {metric_name:20s}: 평가 실패 - {e}")

---
## 3. 질문 난이도별 평가 (핵심)

30개 질문을 **난이도별로 그룹화**하여 RAGAS 평가를 실행합니다.  
GraphRAG의 진가는 **Hard(3-hop) 질문**에서 드러납니다.

```
Easy  (1-hop): "삼성전자 CEO는?"              → 벡터 RAG도 가능
Medium(2-hop): "삼성 투자기관은?"              → GraphRAG 유리
Hard  (3-hop): "삼성 투자기관의 다른 투자처는?" → GraphRAG만 가능
```

In [ ]:
# ============================================================
# 3-1. 난이도별 RAG 파이프라인 실행 (GraphRAG)
# ============================================================
def run_rag_pipeline(questions: list[dict], search_fn, label: str) -> dict:
    """질문 리스트에 대해 RAG 파이프라인을 실행하고 결과를 반환합니다.
    
    Args:
        questions: 질문 딕셔너리 리스트
        search_fn: 검색 함수 (graph_search 또는 vector_search)
        label: 결과 식별 라벨
    Returns:
        dict with 'dataset' (RAGAS용)과 'details' (상세 결과)
    """
    all_questions = []
    all_answers = []
    all_contexts = []
    all_ground_truths = []
    details = []
    
    for i, q in enumerate(questions):
        print(f"  [{label}] {i+1}/{len(questions)}: {q['question'][:40]}...", end=" ")
        
        # 검색
        start_time = time.time()
        contexts = search_fn(q["question"])
        search_time = time.time() - start_time
        
        # 답변 생성
        start_time = time.time()
        answer = generate_answer(q["question"], contexts)
        gen_time = time.time() - start_time
        
        all_questions.append(q["question"])
        all_answers.append(answer)
        all_contexts.append(contexts)
        all_ground_truths.append(q["ground_truth"])
        
        details.append({
            "question": q["question"],
            "answer": answer,
            "ground_truth": q["ground_truth"],
            "difficulty": q["difficulty"],
            "hops": q["hops"],
            "context_count": len(contexts),
            "search_time": search_time,
            "gen_time": gen_time
        })
        
        print(f"({search_time:.1f}s + {gen_time:.1f}s)")
    
    dataset = Dataset.from_dict({
        "question": all_questions,
        "answer": all_answers,
        "contexts": all_contexts,
        "ground_truth": all_ground_truths
    })
    
    return {"dataset": dataset, "details": details}

print("파이프라인 함수 정의 완료")

In [ ]:
# ============================================================
# 3-2. Easy (1-hop) 질문 평가
# ============================================================
print("=" * 60)
print("Easy (1-hop) 질문 10개 — GraphRAG 파이프라인")
print("=" * 60)

easy_results = run_rag_pipeline(easy_qs, graph_search, "Easy")

print("\nRAGAS 평가 중...")
easy_scores = evaluate(
    easy_results["dataset"],
    metrics=[faithfulness, answer_relevancy, context_precision, context_recall]
)
print("\n[Easy 결과]")
for metric, score in easy_scores.items():
    print(f"  {metric}: {score:.4f}")

In [ ]:
# ============================================================
# 3-3. Medium (2-hop) 질문 평가
# ============================================================
print("=" * 60)
print("Medium (2-hop) 질문 10개 — GraphRAG 파이프라인")
print("=" * 60)

medium_results = run_rag_pipeline(medium_qs, graph_search, "Medium")

print("\nRAGAS 평가 중...")
medium_scores = evaluate(
    medium_results["dataset"],
    metrics=[faithfulness, answer_relevancy, context_precision, context_recall]
)
print("\n[Medium 결과]")
for metric, score in medium_scores.items():
    print(f"  {metric}: {score:.4f}")

In [ ]:
# ============================================================
# 3-4. Hard (3-hop) 질문 평가
# ============================================================
print("=" * 60)
print("Hard (3-hop) 질문 10개 — GraphRAG 파이프라인")
print("=" * 60)

hard_results = run_rag_pipeline(hard_qs, graph_search, "Hard")

print("\nRAGAS 평가 중...")
hard_scores = evaluate(
    hard_results["dataset"],
    metrics=[faithfulness, answer_relevancy, context_precision, context_recall]
)
print("\n[Hard 결과]")
for metric, score in hard_scores.items():
    print(f"  {metric}: {score:.4f}")

In [ ]:
# ============================================================
# 3-5. 난이도별 메트릭 비교 DataFrame + 시각화
# ============================================================

# DataFrame 구성
metrics_list = ["faithfulness", "answer_relevancy", "context_precision", "context_recall"]

comparison_data = {
    "Metric": metrics_list,
    "Easy (1-hop)": [easy_scores.get(m, 0) for m in metrics_list],
    "Medium (2-hop)": [medium_scores.get(m, 0) for m in metrics_list],
    "Hard (3-hop)": [hard_scores.get(m, 0) for m in metrics_list],
}

df_comparison = pd.DataFrame(comparison_data)
df_comparison = df_comparison.set_index("Metric")

print("\n=== 난이도별 RAGAS 메트릭 비교 ===")
print(df_comparison.round(4).to_string())
print("\n평균:")
print(df_comparison.mean().round(4).to_string())

In [ ]:
# ============================================================
# 3-6. 히트맵 + 바 차트 시각화
# ============================================================
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# --- 히트맵 ---
ax1 = axes[0]
im = ax1.imshow(df_comparison.values, cmap="RdYlGn", aspect="auto", vmin=0, vmax=1)

ax1.set_xticks(range(len(df_comparison.columns)))
ax1.set_xticklabels(df_comparison.columns, fontsize=11)
ax1.set_yticks(range(len(df_comparison.index)))
ax1.set_yticklabels(df_comparison.index, fontsize=11)

# 셀 값 표시
for i in range(len(df_comparison.index)):
    for j in range(len(df_comparison.columns)):
        val = df_comparison.values[i, j]
        color = "white" if val < 0.5 else "black"
        ax1.text(j, i, f"{val:.3f}", ha="center", va="center",
                fontsize=12, fontweight="bold", color=color)

ax1.set_title("난이도별 RAGAS 메트릭 히트맵", fontsize=14, fontweight="bold")
plt.colorbar(im, ax=ax1, shrink=0.8)

# --- 그룹 바 차트 ---
ax2 = axes[1]
x = np.arange(len(metrics_list))
width = 0.25

bars1 = ax2.bar(x - width, df_comparison["Easy (1-hop)"], width, label="Easy (1-hop)", color="#22c55e")
bars2 = ax2.bar(x, df_comparison["Medium (2-hop)"], width, label="Medium (2-hop)", color="#f59e0b")
bars3 = ax2.bar(x + width, df_comparison["Hard (3-hop)"], width, label="Hard (3-hop)", color="#ef4444")

ax2.set_xlabel("메트릭", fontsize=12)
ax2.set_ylabel("점수", fontsize=12)
ax2.set_title("난이도별 GraphRAG 성능 비교", fontsize=14, fontweight="bold")
ax2.set_xticks(x)
ax2.set_xticklabels(["Faithful", "Relevancy", "Precision", "Recall"], fontsize=10)
ax2.set_ylim(0, 1.1)
ax2.legend(fontsize=10)
ax2.grid(axis="y", alpha=0.3)

plt.tight_layout()
plt.savefig("data/difficulty_comparison.png", dpi=150, bbox_inches="tight")
plt.show()

print("차트 저장: data/difficulty_comparison.png")

---
## 4. Vector RAG vs GraphRAG 비교

**동일한 질문셋**에 대해 두 가지 검색 전략을 비교합니다.

| 검색 방식 | 특징 | 1-hop | 2-hop | 3-hop |
|-----------|------|-------|-------|-------|
| **Vector RAG** | 유사 문서 Top-K 반환 | 잘 됨 | 불안정 | 어려움 |
| **GraphRAG** | 서브그래프 탐색 | 잘 됨 | 잘 됨 | 잘 됨 |

**핵심 가설**: 1-hop에서는 비슷하지만, 3-hop에서 GraphRAG가 압도적으로 우세합니다.

In [ ]:
# ============================================================
# 4-1. Vector RAG 파이프라인 실행 (동일 질문셋)
# ============================================================
print("=" * 60)
print("Vector RAG — 전체 30개 질문")
print("=" * 60)

# Vector RAG로 Easy 질문
print("\n--- Easy ---")
vec_easy = run_rag_pipeline(easy_qs, vector_search, "Vec-Easy")
vec_easy_scores = evaluate(
    vec_easy["dataset"],
    metrics=[faithfulness, answer_relevancy, context_precision, context_recall]
)

# Vector RAG로 Medium 질문
print("\n--- Medium ---")
vec_medium = run_rag_pipeline(medium_qs, vector_search, "Vec-Med")
vec_medium_scores = evaluate(
    vec_medium["dataset"],
    metrics=[faithfulness, answer_relevancy, context_precision, context_recall]
)

# Vector RAG로 Hard 질문
print("\n--- Hard ---")
vec_hard = run_rag_pipeline(hard_qs, vector_search, "Vec-Hard")
vec_hard_scores = evaluate(
    vec_hard["dataset"],
    metrics=[faithfulness, answer_relevancy, context_precision, context_recall]
)

print("\nVector RAG 평가 완료")

In [ ]:
# ============================================================
# 4-2. Vector RAG vs GraphRAG 비교 DataFrame
# ============================================================

def avg_score(scores_dict: dict) -> float:
    """메트릭 딕셔너리의 평균 점수를 계산합니다."""
    vals = [v for v in scores_dict.values() if isinstance(v, (int, float))]
    return sum(vals) / len(vals) if vals else 0

comparison_vs = pd.DataFrame({
    "난이도": ["Easy (1-hop)", "Medium (2-hop)", "Hard (3-hop)"],
    "Vector RAG 평균": [
        avg_score(vec_easy_scores),
        avg_score(vec_medium_scores),
        avg_score(vec_hard_scores)
    ],
    "GraphRAG 평균": [
        avg_score(easy_scores),
        avg_score(medium_scores),
        avg_score(hard_scores)
    ]
})
comparison_vs["차이 (GraphRAG - Vector)"] = comparison_vs["GraphRAG 평균"] - comparison_vs["Vector RAG 평균"]

print("=== Vector RAG vs GraphRAG 비교 ===")
print(comparison_vs.round(4).to_string(index=False))

# 메트릭별 상세 비교
print("\n=== 메트릭별 상세 비교 ===")
detail_rows = []
for diff, v_scores, g_scores in [
    ("Easy", vec_easy_scores, easy_scores),
    ("Medium", vec_medium_scores, medium_scores),
    ("Hard", vec_hard_scores, hard_scores)
]:
    for m in metrics_list:
        detail_rows.append({
            "난이도": diff,
            "메트릭": m,
            "Vector": v_scores.get(m, 0),
            "Graph": g_scores.get(m, 0),
            "차이": g_scores.get(m, 0) - v_scores.get(m, 0)
        })

df_detail = pd.DataFrame(detail_rows)
print(df_detail.round(4).to_string(index=False))

In [ ]:
# ============================================================
# 4-3. 비교 차트: "1-hop 비슷, 3-hop GraphRAG 우세" 패턴 확인
# ============================================================
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

difficulties = ["Easy\n(1-hop)", "Medium\n(2-hop)", "Hard\n(3-hop)"]
x = np.arange(len(difficulties))
width = 0.35

# --- 평균 점수 비교 ---
ax1 = axes[0]
ax1.bar(x - width/2, comparison_vs["Vector RAG 평균"], width,
        label="Vector RAG", color="#94a3b8", edgecolor="#64748b")
ax1.bar(x + width/2, comparison_vs["GraphRAG 평균"], width,
        label="GraphRAG", color="#3b82f6", edgecolor="#1d4ed8")

ax1.set_ylabel("평균 RAGAS 점수", fontsize=12)
ax1.set_title("Vector RAG vs GraphRAG\n난이도별 평균 성능", fontsize=14, fontweight="bold")
ax1.set_xticks(x)
ax1.set_xticklabels(difficulties, fontsize=11)
ax1.set_ylim(0, 1.1)
ax1.legend(fontsize=11)
ax1.grid(axis="y", alpha=0.3)

# 차이 표시 화살표
for i, row in comparison_vs.iterrows():
    diff = row["차이 (GraphRAG - Vector)"]
    if diff > 0:
        ax1.annotate(f"+{diff:.2f}",
                    xy=(i + width/2, row["GraphRAG 평균"]),
                    xytext=(i + width/2, row["GraphRAG 평균"] + 0.05),
                    ha="center", fontsize=10, fontweight="bold", color="#16a34a")

# --- Context Recall 비교 (GraphRAG 강점 메트릭) ---
ax2 = axes[1]
vec_recall = [vec_easy_scores.get("context_recall", 0),
              vec_medium_scores.get("context_recall", 0),
              vec_hard_scores.get("context_recall", 0)]
graph_recall = [easy_scores.get("context_recall", 0),
                medium_scores.get("context_recall", 0),
                hard_scores.get("context_recall", 0)]

ax2.plot(difficulties, vec_recall, "o-", color="#94a3b8", linewidth=2,
         markersize=10, label="Vector RAG", markeredgecolor="#64748b")
ax2.plot(difficulties, graph_recall, "s-", color="#3b82f6", linewidth=2,
         markersize=10, label="GraphRAG", markeredgecolor="#1d4ed8")

ax2.fill_between(range(3), vec_recall, graph_recall,
                 alpha=0.15, color="#3b82f6")

ax2.set_ylabel("Context Recall", fontsize=12)
ax2.set_title("Context Recall 비교\n(GraphRAG 강점 메트릭)", fontsize=14, fontweight="bold")
ax2.set_ylim(0, 1.1)
ax2.legend(fontsize=11)
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.savefig("data/vector_vs_graph_comparison.png", dpi=150, bbox_inches="tight")
plt.show()

print("\n핵심 발견:")
print("  - Easy(1-hop): 두 방식의 차이가 작음")
print("  - Hard(3-hop): GraphRAG가 Context Recall에서 크게 우세")
print("  -> Multi-hop 추론이 필요한 질문에서 GraphRAG의 존재 이유가 드러남")

---
## 5. Neo4j 성능 최적화

프로덕션 환경에서 GraphRAG의 응답 속도를 결정하는 것은 **Neo4j 쿼리 성능**입니다.  
인덱스, APOC, 실행 계획 분석으로 최적화합니다.

In [ ]:
# ============================================================
# 5-1. 인덱스 생성: CREATE INDEX, CREATE FULLTEXT INDEX
# ============================================================
with driver.session() as session:
    # 기존 인덱스 확인
    existing = session.run("SHOW INDEXES YIELD name RETURN collect(name) AS names")
    existing_names = existing.single()["names"]
    print(f"기존 인덱스: {existing_names}")
    
    # --- 노드 프로퍼티 인덱스 ---
    index_commands = [
        # 회사 이름 인덱스 (가장 자주 검색)
        "CREATE INDEX company_name IF NOT EXISTS FOR (c:Company) ON (c.name)",
        # 인물 이름 인덱스
        "CREATE INDEX person_name IF NOT EXISTS FOR (p:Person) ON (p.name)",
        # 제품 이름 인덱스
        "CREATE INDEX product_name IF NOT EXISTS FOR (p:Product) ON (p.name)",
        # 기사 ID 인덱스
        "CREATE INDEX article_id IF NOT EXISTS FOR (a:Article) ON (a.id)",
    ]
    
    for cmd in index_commands:
        try:
            session.run(cmd)
            idx_name = cmd.split("INDEX")[1].split("IF")[0].strip()
            print(f"  인덱스 생성: {idx_name}")
        except Exception as e:
            print(f"  인덱스 스킵 (이미 존재): {e}")
    
    # --- 풀텍스트 인덱스 (자연어 검색용) ---
    try:
        session.run("""
            CREATE FULLTEXT INDEX fulltext_index IF NOT EXISTS
            FOR (n:Company|Person|Product|Article)
            ON EACH [n.name, n.title, n.content, n.description]
        """)
        print("  풀텍스트 인덱스 생성: fulltext_index")
    except Exception as e:
        print(f"  풀텍스트 인덱스 스킵: {e}")
    
    # 최종 인덱스 목록
    result = session.run("SHOW INDEXES YIELD name, type, labelsOrTypes, properties RETURN *")
    print("\n=== 현재 인덱스 목록 ===")
    for record in result:
        print(f"  {record['name']:30s} | {record['type']:15s} | {record['labelsOrTypes']} -> {record['properties']}")

In [ ]:
# ============================================================
# 5-2. 인덱스 전/후 쿼리 시간 비교 (10회 반복 측정)
# ============================================================
def benchmark_query(session, query: str, params: dict = None, n_runs: int = 10) -> list[float]:
    """쿼리를 n_runs번 실행하고 각 실행 시간(ms)을 반환합니다."""
    times = []
    for _ in range(n_runs):
        start = time.perf_counter()
        result = session.run(query, params or {})
        _ = list(result)  # 결과 소비 (실제 실행 보장)
        elapsed = (time.perf_counter() - start) * 1000  # ms
        times.append(elapsed)
    return times

# 벤치마크 쿼리들
benchmark_queries = [
    {
        "name": "단일 노드 조회 (이름)",
        "query": "MATCH (c:Company {name: $name}) RETURN c",
        "params": {"name": "삼성전자"}
    },
    {
        "name": "1-hop 이웃 탐색",
        "query": "MATCH (c:Company {name: $name})-[r]-(n) RETURN c, type(r), n",
        "params": {"name": "삼성전자"}
    },
    {
        "name": "2-hop 경로 탐색",
        "query": "MATCH path = (c:Company {name: $name})-[*1..2]-(n) RETURN path LIMIT 20",
        "params": {"name": "삼성전자"}
    },
    {
        "name": "전체 노드 카운트",
        "query": "MATCH (n) RETURN count(n) AS cnt",
        "params": {}
    }
]

print("=== 쿼리 성능 벤치마크 (10회 반복) ===")
bench_results = []

with driver.session() as session:
    for bq in benchmark_queries:
        try:
            times = benchmark_query(session, bq["query"], bq["params"])
            avg_ms = sum(times) / len(times)
            min_ms = min(times)
            max_ms = max(times)
            p50 = sorted(times)[len(times)//2]
            
            bench_results.append({
                "쿼리": bq["name"],
                "평균(ms)": round(avg_ms, 2),
                "최소(ms)": round(min_ms, 2),
                "최대(ms)": round(max_ms, 2),
                "P50(ms)": round(p50, 2)
            })
            print(f"  {bq['name']:25s}: 평균 {avg_ms:.2f}ms (min={min_ms:.2f}, max={max_ms:.2f})")
        except Exception as e:
            print(f"  {bq['name']:25s}: 실행 실패 - {e}")

df_bench = pd.DataFrame(bench_results)
print("\n")
print(df_bench.to_string(index=False))

In [ ]:
# ============================================================
# 5-3. APOC 프로시저 활용: 배치 처리
# ============================================================
print("=== APOC 배치 처리 예시 ===")
print("""  
# 대량 데이터 배치 업데이트 (1000건씩)
# 프로덕션에서 수만 건 이상 처리 시 필수

CALL apoc.periodic.iterate(
  "MATCH (c:Company) RETURN c",
  "SET c.updated_at = datetime()",
  {batchSize: 1000, parallel: true}
)
""")

# APOC 사용 가능 여부 확인
with driver.session() as session:
    try:
        result = session.run("RETURN apoc.version() AS version")
        version = result.single()["version"]
        print(f"APOC 버전: {version}")
        
        # 배치 처리 데모 (읽기 전용 — 안전)
        result = session.run("""
            CALL apoc.periodic.iterate(
                "MATCH (n) RETURN n LIMIT 100",
                "RETURN count(n)",
                {batchSize: 10, parallel: false}
            )
            YIELD batches, total, timeTaken
            RETURN batches, total, timeTaken
        """)
        for record in result:
            print(f"  배치 수: {record['batches']}, 총 처리: {record['total']}, 소요: {record['timeTaken']}s")
    except Exception as e:
        print(f"APOC 미설치 또는 비활성: {e}")
        print("  -> docker-compose.yml에서 NEO4J_PLUGINS: '[\"apoc\"]' 확인")

In [ ]:
# ============================================================
# 5-4. EXPLAIN/PROFILE로 쿼리 실행 계획 분석
# ============================================================
print("=== 쿼리 실행 계획 분석 ===")

with driver.session() as session:
    # PROFILE: 실제 실행하면서 각 단계 통계 수집
    profile_query = """
    PROFILE
    MATCH (c:Company {name: "삼성전자"})-[r]-(neighbor)
    RETURN c.name, type(r) AS rel_type, labels(neighbor) AS neighbor_labels,
           neighbor.name AS neighbor_name
    """
    
    try:
        result = session.run(profile_query)
        records = list(result)
        
        # 실행 결과
        print(f"\n결과 행 수: {len(records)}")
        for rec in records[:5]:
            print(f"  {rec['c.name']} -[{rec['rel_type']}]-> "
                  f"{rec['neighbor_labels']}: {rec['neighbor_name']}")
        
        # 프로파일 정보
        summary = result.consume()
        if hasattr(summary, 'profile') and summary.profile:
            profile = summary.profile
            print(f"\n실행 계획:")
            print(f"  DB Hits: {profile.get('dbHits', 'N/A')}")
            print(f"  Rows: {profile.get('rows', 'N/A')}")
            print(f"  Plan: {profile.get('operatorType', 'N/A')}")
    except Exception as e:
        print(f"PROFILE 실행 실패: {e}")
        print("  -> 일반 MATCH 쿼리로 대체")

# 실행 계획 해석 가이드
print("\n=== 실행 계획 해석 가이드 ===")
print("""
| 연산자           | 의미                        | 최적화 포인트       |
|------------------|----------------------------|--------------------|
| NodeByLabelScan  | 라벨 전체 스캔              | 인덱스 추가 필요    |
| NodeIndexSeek    | 인덱스 활용 (좋음)          | 최적 상태           |
| Filter           | 결과 필터링                 | WHERE 조건 최적화   |
| Expand(All)      | 관계 탐색                   | depth 제한 확인     |
| CartesianProduct | 교차곱 (위험!)              | MATCH 패턴 재설계   |

핵심: NodeByLabelScan → NodeIndexSeek으로 바꾸면 성능 10배 이상 향상
""")

In [ ]:
# ============================================================
# 5-5. 메모리 설정 권장값
# ============================================================
print("=== Neo4j 메모리 설정 권장값 ===")
print("""
# neo4j.conf 또는 docker-compose.yml 환경변수

# ---- 개발 환경 (노드 ~10K) ----
NEO4J_server_memory_heap_initial__size=512m
NEO4J_server_memory_heap_max__size=1g
NEO4J_server_memory_pagecache_size=512m

# ---- 프로덕션 환경 (노드 ~1M) ----
NEO4J_server_memory_heap_initial__size=4g
NEO4J_server_memory_heap_max__size=4g
NEO4J_server_memory_pagecache_size=8g

# ---- 대규모 환경 (노드 ~10M+) ----
NEO4J_server_memory_heap_initial__size=8g
NEO4J_server_memory_heap_max__size=8g
NEO4J_server_memory_pagecache_size=16g

# 권장 공식:
#   Page Cache = 데이터 크기 * 1.2
#   Heap = min(31g, 서버 메모리 * 0.25)
#   나머지 = OS + 파일 시스템 캐시
""")

# 현재 Neo4j 상태 확인
with driver.session() as session:
    try:
        result = session.run("""
            CALL dbms.components() YIELD name, versions, edition
            RETURN name, versions, edition
        """)
        for record in result:
            print(f"현재 Neo4j: {record['name']} {record['versions']} ({record['edition']})")
    except Exception:
        pass
    
    # 그래프 크기 확인
    try:
        result = session.run("""
            MATCH (n) WITH count(n) AS nodes
            MATCH ()-[r]->() WITH nodes, count(r) AS rels
            RETURN nodes, rels
        """)
        record = result.single()
        if record:
            print(f"현재 그래프 크기: 노드 {record['nodes']}개, 관계 {record['rels']}개")
    except Exception as e:
        print(f"그래프 크기 확인 실패: {e}")

---
## 6. 프로덕션 체크리스트

GraphRAG를 프로덕션에 배포하기 위한 **4가지 핵심 영역**을 점검합니다.

### 6-1. 데이터 파이프라인

```
소스 데이터 → [추출] → [정제] → [적재] → [인덱싱] → Neo4j
    |            |         |         |          |
    PDF/HTML     LLM       ER        Cypher     인덱스
    DB/API       VLM       검증      배치처리   풀텍스트
```

| 단계 | 도구 | 주의점 |
|------|------|--------|
| 추출 | GPT-4o, Claude | 프롬프트 버전 관리, 비용 모니터링 |
| 정제 | Entity Resolution (Part 4) | 중복 임계값 튜닝 (0.85 권장) |
| 적재 | neo4j Python driver | 배치 UNWIND, 트랜잭션 관리 |
| 인덱싱 | CREATE INDEX | 라벨별 프로퍼티 인덱스 필수 |

### 6-2. 모니터링

| 모니터링 대상 | 도구 | 임계값 |
|---------------|------|--------|
| 쿼리 성능 | Neo4j Browser / LangSmith | P95 < 500ms |
| 그래프 크기 | `MATCH (n) RETURN count(n)` | 주간 추적 |
| LLM API 비용 | OpenAI Usage Dashboard | 일일 예산 설정 |
| RAGAS 점수 | 주기적 평가 (주 1회) | 평균 > 0.7 |
| 에러율 | 애플리케이션 로그 | < 1% |

### 6-3. CI/CD

```yaml
# .github/workflows/graphrag-ci.yml
name: GraphRAG CI
on: [push, pull_request]
jobs:
  test:
    runs-on: ubuntu-latest
    services:
      neo4j:
        image: neo4j:5-community
        env:
          NEO4J_AUTH: neo4j/testpassword
          NEO4J_PLUGINS: '["apoc"]'
        ports: ["7687:7687"]
    steps:
      - uses: actions/checkout@v4
      - run: pip install -r requirements.txt
      - run: python -m pytest tests/ -v
      - run: python eval/run_ragas.py --threshold 0.7
```

**스키마 변경 관리:**
- 스키마 변경은 마이그레이션 스크립트로 관리
- `schema/v1.cypher`, `schema/v2.cypher` 형태로 버전 관리
- 테스트 환경에서 먼저 적용 후 프로덕션 배포

### 6-4. 보안

| 항목 | 설정 | 이유 |
|------|------|------|
| 읽기 전용 사용자 | `GRANT READ ON DATABASE * TO reader` | API 서버에서 사용 |
| 쿼리 검증 | Cypher injection 방지 (파라미터화) | Text2Cypher 출력 검증 |
| API 키 관리 | 환경변수 / Secret Manager | `.env` 절대 커밋 금지 |
| 네트워크 | VPC 내부에서만 Neo4j 접근 | 7687 포트 외부 차단 |

In [ ]:
# ============================================================
# 6-5. 프로덕션 체크리스트 자가진단 스크립트
# ============================================================
def production_checklist():
    """프로덕션 배포 전 자동 체크리스트를 실행합니다."""
    checks = []
    
    # 1. Neo4j 연결 확인
    try:
        with driver.session() as session:
            session.run("RETURN 1")
        checks.append(("Neo4j 연결", "PASS", "정상 연결"))
    except Exception as e:
        checks.append(("Neo4j 연결", "FAIL", str(e)))
    
    # 2. 인덱스 존재 확인
    try:
        with driver.session() as session:
            result = session.run("SHOW INDEXES YIELD name RETURN count(*) AS cnt")
            cnt = result.single()["cnt"]
            status = "PASS" if cnt >= 3 else "WARN"
            checks.append(("인덱스", status, f"{cnt}개 인덱스"))
    except Exception:
        checks.append(("인덱스", "FAIL", "확인 불가"))
    
    # 3. 그래프 데이터 존재 확인
    try:
        with driver.session() as session:
            result = session.run("MATCH (n) RETURN count(n) AS cnt")
            cnt = result.single()["cnt"]
            status = "PASS" if cnt > 0 else "FAIL"
            checks.append(("그래프 데이터", status, f"노드 {cnt}개"))
    except Exception:
        checks.append(("그래프 데이터", "FAIL", "확인 불가"))
    
    # 4. OpenAI API 키 확인
    api_key = os.getenv("OPENAI_API_KEY", "")
    if api_key and len(api_key) > 10:
        checks.append(("OpenAI API 키", "PASS", f"...{api_key[-4:]}"))
    else:
        checks.append(("OpenAI API 키", "FAIL", "미설정"))
    
    # 5. APOC 플러그인
    try:
        with driver.session() as session:
            result = session.run("RETURN apoc.version() AS v")
            v = result.single()["v"]
            checks.append(("APOC 플러그인", "PASS", f"v{v}"))
    except Exception:
        checks.append(("APOC 플러그인", "WARN", "미설치 (선택사항)"))
    
    # 6. 쿼리 성능
    try:
        with driver.session() as session:
            start = time.perf_counter()
            session.run("MATCH (n) RETURN n LIMIT 100").consume()
            elapsed_ms = (time.perf_counter() - start) * 1000
            status = "PASS" if elapsed_ms < 100 else ("WARN" if elapsed_ms < 500 else "FAIL")
            checks.append(("쿼리 성능 (100노드)", status, f"{elapsed_ms:.1f}ms"))
    except Exception:
        checks.append(("쿼리 성능", "FAIL", "측정 불가"))
    
    # 결과 출력
    print("=" * 60)
    print("  프로덕션 배포 체크리스트")
    print("=" * 60)
    
    pass_count = 0
    for name, status, detail in checks:
        icon = {"PASS": "[OK]", "WARN": "[!!]", "FAIL": "[XX]"}[status]
        print(f"  {icon} {name:25s} {detail}")
        if status == "PASS":
            pass_count += 1
    
    print(f"\n  결과: {pass_count}/{len(checks)} 통과")
    if pass_count == len(checks):
        print("  -> 프로덕션 배포 준비 완료!")
    else:
        print("  -> 위 항목을 확인 후 배포하세요.")

production_checklist()

---
## 7. 최종 아키텍처 요약

### Part 1~7 전체 파이프라인

```
 [Part 1] 동기 부여                    [Part 7] 프로덕션
    |                                      ^
    v                                      |
 [Part 2] 수작업 KG    ──────────>    RAGAS 평가
    |                                 Neo4j 최적화
    v                                 CI/CD + 모니터링
 [Part 3] LLM 자동 추출                    ^
    |                                      |
    v                                      |
 [Part 4] Entity Resolution ────>   정제된 KG
    |                                      |
    v                                      |
 [Part 5] 멀티모달 VLM ─────────>   통합 KG (텍스트+표+이미지)
    |                                      |
    v                                      v
 [Part 6] 검색 파이프라인 ──────>   GraphRAG 시스템 완성
           Text2Cypher Agent
           하이브리드 검색
           답변 생성
```

### 각 Part 핵심 기술 정리

| Part | 핵심 | 기술 스택 | Milestone |
|------|------|-----------|------------|
| **1** | 왜 GraphRAG인가 | Neo4j, Cypher 기초 | 첫 그래프 생성 (7노드) |
| **2** | 수작업 KG 구축 | 온톨로지, Meta-Dictionary | 수작업 KG (15노드, 20관계) |
| **3** | LLM 자동 추출 | GPT-4o, Structured Output | 자동 KG 생성 |
| **4** | Entity Resolution | RapidFuzz, 임베딩 유사도 | 중복 제거 완료 |
| **5** | 멀티모달 VLM | GPT-4o Vision, 표 파싱 | 텍스트+표 통합 KG |
| **6** | 검색 파이프라인 | Text2Cypher, 하이브리드 | GraphRAG 시스템 완성 |
| **7** | 평가 + 프로덕션 | RAGAS, 인덱스, CI/CD | 프로덕션 배포 준비 |

### 다음 단계 추천

이 커리큘럼을 마친 후 도전할 수 있는 고급 주제:

1. **고급 그래프 알고리즘** — PageRank, Betweenness Centrality로 핵심 노드 발견
2. **커뮤니티 탐지** — Louvain, Label Propagation으로 클러스터링
3. **GNN (Graph Neural Network)** — PyG, DGL로 그래프 기반 딥러닝
4. **Microsoft GraphRAG** — 글로벌 검색 + 커뮤니티 요약
5. **멀티 도메인 통합** — 여러 도메인 그래프를 통합하는 전사 KG

In [ ]:
# ============================================================
# 7-1. 전체 커리큘럼 완료 요약 출력
# ============================================================
summary = """
============================================================
   GraphRAG 실습 커리큘럼 — 전체 완료 요약
============================================================

Part 1: 왜 GraphRAG인가?
  -> Neo4j에 첫 그래프 생성 완료 (노드 7개 + 관계 5개)

Part 2: 수작업 KG 구축
  -> 온톨로지 설계 + Meta-Dictionary 작성 완료

Part 3: LLM 자동 추출
  -> GPT-4o로 뉴스 10건에서 자동 KG 생성 완료

Part 4: Entity Resolution
  -> RapidFuzz + 임베딩으로 중복 엔티티 해소 완료

Part 5: 멀티모달 VLM
  -> 표/이미지 문서를 그래프로 변환 완료

Part 6: 통합 + 검색
  -> Text2Cypher Agent + 하이브리드 검색 시스템 완성

Part 7: 평가 + 프로덕션 (현재)
  -> RAGAS 평가 + Neo4j 최적화 + 프로덕션 체크리스트 완료

============================================================
  핵심 메시지 7줄
============================================================

  1. 문제 정의가 먼저 — GraphRAG부터 시작하지 마라
  2. 암묵지를 Meta-Dictionary로 체계화
  3. 표는 SQL, 문서는 계층 — 각각 다르게 접근
  4. 가중치 싸움이 디자인을 결정
  5. Text2Cypher = 삽질의 연속 -> Agent로 해결
  6. 1-hop이면 벡터로 충분 — Multi-hop이 존재 이유
  7. 정답은 없다 — PoC, 상황별 선택, 교차 평가

============================================================
  수고하셨습니다! 이제 여러분의 프로젝트에 적용해보세요.
============================================================
"""
print(summary)

---
## 8. 연습 문제

### 연습 8-1: 자신만의 평가 질문 5개 추가

아래 템플릿을 사용하여 **본인 도메인에 맞는 평가 질문 5개**를 작성하세요.  
난이도를 골고루 포함하는 것이 좋습니다 (Easy 1, Medium 2, Hard 2).

In [ ]:
# ============================================================
# 연습 8-1: 자신만의 평가 질문 작성
# ============================================================

# TODO: 아래 질문을 본인 도메인에 맞게 수정하세요
my_questions = [
    {
        "question": "여기에 Easy 질문을 작성하세요",
        "ground_truth": "정답",
        "difficulty": "easy",
        "hops": 1
    },
    {
        "question": "여기에 Medium 질문을 작성하세요 (1)",
        "ground_truth": "정답",
        "difficulty": "medium",
        "hops": 2
    },
    {
        "question": "여기에 Medium 질문을 작성하세요 (2)",
        "ground_truth": "정답",
        "difficulty": "medium",
        "hops": 2
    },
    {
        "question": "여기에 Hard 질문을 작성하세요 (1)",
        "ground_truth": "정답",
        "difficulty": "hard",
        "hops": 3
    },
    {
        "question": "여기에 Hard 질문을 작성하세요 (2)",
        "ground_truth": "정답",
        "difficulty": "hard",
        "hops": 3
    }
]

# 작성 후 아래 코드로 검증
for i, q in enumerate(my_questions):
    print(f"{i+1}. [{q['difficulty']:6s}] {q['question']}")
    print(f"   정답: {q['ground_truth']}")

# 선택: RAGAS 평가 실행
# my_results = run_rag_pipeline(my_questions, graph_search, "Custom")
# my_scores = evaluate(my_results["dataset"], metrics=[faithfulness, answer_relevancy])
# print(my_scores)

### 연습 8-2: 인덱스 전략 실험

다양한 인덱스 전략을 실험하고, 쿼리 성능 변화를 측정하세요.

In [ ]:
# ============================================================
# 연습 8-2: 인덱스 전략 실험
# ============================================================

# TODO: 다양한 인덱스를 만들고 벤치마크를 비교하세요

# 실험 1: 복합 인덱스 (여러 프로퍼티)
# with driver.session() as session:
#     session.run("""
#         CREATE INDEX company_name_sector IF NOT EXISTS
#         FOR (c:Company) ON (c.name, c.sector)
#     """)

# 실험 2: 관계 인덱스 (Neo4j 5.7+)
# with driver.session() as session:
#     session.run("""
#         CREATE INDEX rel_invested IF NOT EXISTS
#         FOR ()-[r:INVESTED_IN]-() ON (r.amount)
#     """)

# 실험 3: 인덱스 유무에 따른 벤치마크 비교
# 힌트: benchmark_query() 함수를 활용하세요

print("인덱스 전략 실험 셀입니다.")
print("위의 주석을 해제하고 실행하여 실험해보세요.")
print("")
print("실험 포인트:")
print("  1. 단일 프로퍼티 인덱스 vs 복합 인덱스")
print("  2. 풀텍스트 인덱스의 한국어 지원 확인")
print("  3. PROFILE로 인덱스 사용 여부 확인")

### 연습 8-3: End-to-End 파이프라인 테스트

Part 1~7 전체 파이프라인을 하나의 함수로 통합하고 테스트하세요.

In [ ]:
# ============================================================
# 연습 8-3: End-to-End 파이프라인 테스트
# ============================================================

def end_to_end_graphrag(question: str, verbose: bool = True) -> dict:
    """GraphRAG 전체 파이프라인을 실행합니다.
    
    1. 질문 분석 (난이도 추정)
    2. 검색 (그래프 탐색)
    3. 답변 생성
    4. 자체 품질 점검
    
    Returns:
        dict: question, answer, contexts, metadata
    """
    start_total = time.time()
    
    # 1단계: 질문 분석
    if verbose:
        print(f"질문: {question}")
        print("---")
    
    # 2단계: 그래프 검색
    t0 = time.time()
    contexts = graph_search(question, depth=2)
    search_time = time.time() - t0
    
    if verbose:
        print(f"[검색] {len(contexts)}개 문맥 ({search_time:.2f}s)")
    
    # 3단계: 답변 생성
    t0 = time.time()
    answer = generate_answer(question, contexts)
    gen_time = time.time() - t0
    
    if verbose:
        print(f"[생성] {gen_time:.2f}s")
        print(f"답변: {answer}")
    
    # 4단계: 자체 품질 점검
    total_time = time.time() - start_total
    
    result = {
        "question": question,
        "answer": answer,
        "contexts": contexts,
        "metadata": {
            "context_count": len(contexts),
            "search_time_s": round(search_time, 3),
            "gen_time_s": round(gen_time, 3),
            "total_time_s": round(total_time, 3)
        }
    }
    
    if verbose:
        print(f"\n총 소요시간: {total_time:.2f}s")
    
    return result


# 테스트 실행
print("=== End-to-End 테스트 ===")
print()

test_questions = [
    "삼성전자가 양산을 시작한 차세대 메모리는?",
    "네이버와 삼성전자가 공동 개발한 기술이 적용될 기기는?",
    "반도체 기업 두 곳의 경쟁 관계를 설명하면?"
]

for q in test_questions:
    print("=" * 50)
    result = end_to_end_graphrag(q)
    print()

In [ ]:
# ============================================================
# 정리: Neo4j 연결 종료
# ============================================================
driver.close()
print("Neo4j 연결 종료")
print()
print("=" * 60)
print("  Part 7 완료 — GraphRAG 실습 커리큘럼을 마칩니다.")
print("  수고하셨습니다!")
print("=" * 60)